In [1]:
import heeps
import numpy as np
from astropy.io import fits

In [2]:
conf = dict(
    dir_current = '$HOME/heeps_metis',  # specify a directory
    nframes = 100,                      # number of SCAO phase screens selected
    f_phase = 'wavefront/COMPASS_201810_RandomWind_100screens_meters.fits',
    f_pupil = 'pupil/ELT_fullM1.fits',  # entrance pupil file
)
conf = heeps.config.read_config(verbose=False, **conf)
conf = heeps.config.update_config(saveconf=True, verbose=True, **conf)
phase_screens, amp_screens, tiptilts, misaligns = heeps.wavefront.load_errors(
    verbose=True, **conf)

Simulation config: band=L, mode=RAVC
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
   loading vortex back-propagation params
   vc_charge=2, ngrid=1024, beam_ratio=0.2568
   ravc_calc=True, ravc_t=0.7608, ravc_r=0.6213
   npupil=285, pscale=5.47 mas, lam=3.8112E-06 m
   hfov=1.1 arcsec (-> ndet=403, 51.74 lam/D)

Load phase screens from 'COMPASS_201810_RandomWind_100screens_meters.fits'
   nscreens=100 (nframes=100, nstep=1)


In [3]:
lams = np.linspace(3.7, 3.9, 5)*1e-6
lam0 = conf['lam']
pupil_img_size0 = conf['pupil_img_size']
verbose = False
for i, conf['lam'] in enumerate(lams):
    conf['pupil_img_size'] = pupil_img_size0*conf['lam']/lam0
    wf = heeps.pupil.pupil(savefits=True, verbose=verbose, **conf)
    psf = heeps.wavefront.propagate_one(wf, onaxis=False, savefits=True, 
        verbose=verbose, **conf)
    psf = np.expand_dims(psf, axis=-3)
    psfs = heeps.wavefront.propagate_cube(wf, phase_screens=phase_screens,
        amp_screens=amp_screens, tiptilts=tiptilts, misaligns=misaligns, 
        onaxis=True, savefits=True, verbose=True, **conf)
    psfs = np.expand_dims(psfs, axis=-3)
    if i == 0:
        offaxis_psf = psf
        onaxis_psfs = psfs
    else:
        offaxis_psf = np.concatenate((offaxis_psf, psf), axis=-3)
        onaxis_psfs = np.concatenate((onaxis_psfs, psfs), axis=-3)
fits.writeto('chrom_offaxis_psf.fits', offaxis_psf, overwrite=True)
fits.writeto('chrom_onaxis_psfs.fits', onaxis_psfs, overwrite=True)

Create on-axis PSF cube
   2022-03-04 05:08:12, e2e simulation using 12 cores
   2022-03-04 05:08:54, completed in 42.19 seconds
Create on-axis PSF cube
   2022-03-04 05:08:56, e2e simulation using 12 cores
   2022-03-04 05:09:39, completed in 43.62 seconds
Create on-axis PSF cube
   2022-03-04 05:09:41, e2e simulation using 12 cores
   2022-03-04 05:10:25, completed in 44.38 seconds
Create on-axis PSF cube
   2022-03-04 05:10:27, e2e simulation using 12 cores
   2022-03-04 05:11:11, completed in 43.75 seconds
Create on-axis PSF cube
   2022-03-04 05:11:13, e2e simulation using 12 cores
   2022-03-04 05:11:58, completed in 45.07 seconds


In [4]:
print(offaxis_psf.shape)
print(onaxis_psfs.shape)

(5, 403, 403)
(100, 5, 403, 403)
